In [78]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [51]:
#raw_data
df1 = pd.read_csv("Bike_Share_DF_CLEANED_1.csv")
df2 = pd.read_csv("Bike_Share_DF_CLEANED_2.csv")
df3 = pd.read_csv("Bike_Share_DF_CLEANED_3.csv")
df4 = pd.read_csv("Bike_Share_DF_CLEANED_4.csv")
df5 = pd.read_csv("Bike_Share_DF_CLEANED_5.csv")
df6 = pd.read_csv("Bike_Share_DF_CLEANED_6.csv")
df7 = pd.read_csv("Bike_Share_DF_CLEANED_7.csv")

In [58]:

df = df1.append([df2,df3,df4,df5,df6,df7])
# considering 2 years of data
df = df[df['year'].isin(['2014','2015'])]

array([2015, 2014], dtype=int64)

In [59]:
# Prediction for rides for riverside lamar : 2575
df_rs_start = df[df['start_station_id'] == 2575]

## Aggregating the number of rides that starting at 2575 by hour onany given day
df_rs_start_agg = pd.DataFrame(df_rs_start.groupby(['start_station_id','start_station_name','Date','hour'])['trip_id'].\
                               count().reset_index())

df_rs_end = df[df['end_station_id'] == 2575]

## Aggregating the number of rides that starting at 2575 by hour onany given day
df_rs_end_agg = pd.DataFrame(df_rs_end.groupby(['end_station_id','end_station_name','Date','hour'])['trip_id'].\
                               count().reset_index())

df_rs_merged = df_rs_start_agg.merge(df_rs_end_agg,on = ['Date','hour'],suffixes = ['_start','_end'])


In [60]:
df_rs_merged['diff'] = df_rs_merged['trip_id_start'] - df_rs_merged['trip_id_end']

In [81]:
df_rs_temp = df_rs_merged[['start_station_id','start_station_name','Date','hour','trip_id_start']]
df_date_info = df[['Date','month','weekday','TempAvgF','WindAvgMPH','Rain']].drop_duplicates()

In [82]:
df_rs_final = df_rs_temp.merge(df_date_info,on = ['Date'])

In [83]:
df_rs_model_data = df_rs_final.drop(['start_station_id', 'start_station_name','Date'], axis=1)

df_rs_model_data['hour'] = df_rs_model_data['hour'].astype('category')
df_rs_model_data['weekday'] = df_rs_model_data['weekday'].astype('category')
df_rs_model_data['month'] = df_rs_model_data['month'].astype('category')
df_rs_model_data['Rain'] = df_rs_model_data['Rain'].astype('category')

In [ ]:
df_rs_fina

In [84]:
y, X = dmatrices('trip_id_start ~ hour + month + weekday + TempAvgF + WindAvgMPH + Rain', data=df_rs_model_data, return_type='dataframe')

In [86]:
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print result.summary()


                            OLS Regression Results                            
Dep. Variable:          trip_id_start   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     15.00
Date:                Sun, 12 Aug 2018   Prob (F-statistic):           4.66e-98
Time:                        17:53:41   Log-Likelihood:                -7931.0
No. Observations:                3455   AIC:                         1.595e+04
Df Residuals:                    3411   BIC:                         1.622e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         1.0206      0.686      1.487